#  Juliaで高速なベクトル・行列の区間演算をしたい

ベクトルの内積、行列ベクトル積、行列積の区間演算はとても重要。すなわち
$x, y\in\mathbb{IF}^n$, $A, B\in\mathbb{IF}^{n\times n}$ に対して、それぞれ$x^Ty \subset \alpha$, $Ax\subset z$, $AB\subset C$となる
$$
\alpha\in\mathbb{IF},\quad z\in\mathbb{IF}^n,\quad C\in\mathbb{IF}^{n\times n}
$$
を求める方法を考える。

## 素朴な方法

最も素朴な方法は各演算を区間演算に変更した方式である。

$$
    \alpha = \sum_{k = 1}^n x_ky_k
$$

$$
    z_i = \sum_{k = 1}^n A_{i,k}x_k\quad (1\le i\le n)
$$

$$
    C_{ij} = \sum_{k+1}^n A_{i,k}B_{k,j}\quad (1\le i,j\le n)
$$

特にJuliaの`IntervalArithmetic.jl`パッケージでは、この方法で区間演算が行なわれている。しかし、この方法だとプログラムの最適化が難しく、計算機の性能を十分に引き出した実装が難しく、計算時間がかかってしまう。

In [3]:
using IntervalArithmetic, BenchmarkTools

n = 1000;
x = randn(n);
y = randn(n);

x_int = map(Interval, x);
y_int = map(Interval, y);

@btime alpha = dot(x_int,y_int)

# setrounding(Interval, :fast)

# function dot_vec()
#     n = 1000; x = randn(n); y = randn(n);
#     x_int = map(Interval, x); y_int = map(Interval, y);
#     alpha = dot(x_int,y_int);
# end
# @btime dot_vec()

  69.187 μs (6007 allocations: 93.88 KiB)


In [5]:
using LinearAlgebra
n = 1000; A = randn(n,n); x = randn(n); 
A_int = map(Interval, A); x_int = map(Interval, x); y = similar(x_int);
@time z = A_int*x_int;
@time mul!(y,A_int,x_int)
println(y ⊆ z)
println(z ⊆ y)

# @test z ⊆ y
# @btime z = A_int*x_int;

  0.046672 seconds (1 allocation: 15.750 KiB)
  0.045570 seconds
true
true


In [8]:
n = 100;
A = randn(n,n);
B = randn(n,n);
A_int = map(Interval, A);
B_int = map(Interval, B);
@time C = A_int*B_int;
@time C = A_int*B_int;
@time C = A_int*B_int;

  0.048638 seconds (8 allocations: 156.656 KiB)
  0.046084 seconds (8 allocations: 156.656 KiB)
  0.046330 seconds (8 allocations: 156.656 KiB)


In [41]:
n = 1000; A = randn(n,n); B = randn(n,n);
A_int = map(Interval, A); B_int = map(Interval, B);
@btime C = A_int*B_int;

  41.813 s (8 allocations: 15.26 MiB)


これを高速化したい。考えられる方法は

1. 並列化したい（[Loopvectrization](https://github.com/chriselrod/LoopVectorization.jl), [Tullio](https://github.com/mcabbott/Tullio.jl)?, @simd命令）
1. [StaticArrays](https://github.com/JuliaArrays/StaticArrays.jl_)を使うと[速いらしい](https://qiita.com/cometscome_phys/items/669273a49ab2417d3af8)

一つ目はまだ未実験、二つ目は小さいサイズでは多少計算時間が変わるが、要素数が多くなるほどコンパイルに時間がかかり、全体の経過時間は長くなってしまった。要素数が100以下なら`StaticArrays`が使えると上記Webサイトに書いてありました。

そこで残るは

3. BLASを使う区間演算を実装する、だけど丸め方向が変えられないので、丸めの向きを変えない区間演算
    - 内積は森倉定理8あるいは教科書の定理2.4(p.38)
    - 行列ベクトル積は？
    - 行列積は森倉4節
 
 
 ## BLASを使う
 
 まずJuliaの中で、[BLAS](https://ja.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms)を使うには``LinearAlgebra``パッケージが必要。

In [9]:
using LinearAlgebra, BenchmarkTools
# versioninfo()
# BLAS.vendor()
n = 5000;
A, B, C = randn(n,n), randn(n,n), zeros(n,n);

@btime C = A*B;
@btime mul!($C, $A, $B);

  755.193 ms (2 allocations: 190.73 MiB)
  816.399 ms (0 allocations: 0 bytes)


このようにBLASを使うと、物凄いスピードで数値計算ができる。ただし、行列の内部の型がfloat64に限られるのと、丸め方向を変更した計算ができないため、区間演算の結果が粗くなる（区間幅が増加する）。

**定義**　$\mathbf{u}=2^{-53}$を倍精度の単位相対丸めとする。$\mathbf{S}_{\min}=2^{-1074}$を倍精度浮動小数点数の正の最小数とする。$\mathbf{F}_{\min}=2^{-1022}$を倍精度浮動小数点数の正規化数の正の最小数とする。 

**注意**　単位相対丸めは$1$と$1$よりも小さい最大の浮動小数点数との差を表す。つまり
$$
1-\mathbf{u}<a<1
$$
となる$a\in\mathbb{F}$は存在しない。

**注意**　単位相対丸めとは別に計算機イプシロン（unit roundoff, machine epsilon）という単位もあり、こちらは$1$と$1$よりも大きい最小の浮動小数点数との差を表し、64bit浮動小数点数では$2^{-52}$でこちらを$\mathbf{u}$と書く流儀もある（こっちが主流？）ので、注意が必要。Juliaでは`eps(Float64)`とすると数値が得られる。

In [11]:
u = 2.0^-53;
s_min = 2.0^-1074;
f_min = 2.0^-1022;
println(bitstring(1.0))
println(bitstring(1.0-u))
println(bitstring(s_min))
println(bitstring(f_min))

0011111111110000000000000000000000000000000000000000000000000000
0011111111101111111111111111111111111111111111111111111111111111
0000000000000000000000000000000000000000000000000000000000000001
0000000000010000000000000000000000000000000000000000000000000000


In [2]:
function [C_mid,C_rad]=mm_ufp(A_mid,A_rad,B_mid,B_rad)
%現在の丸めモードは最近点であると仮定

    %system_dependent('setround',0.5)
    u=2^-53;
    n=length(A_mid);

    if(2*(n+2)*u>=1)
        error('mm_ufp failed!(2(n+2)u>=1)'),
    end
    
    C_mid = A_mid*B_mid;
    C_rad = (n+2)*u*ufp(abs(A_mid)*abs(B_mid))+realmin;
    
    AmBr=abs(A_mid)*B_rad;
    AmBr=succ_near(AmBr+((n+2)*u*ufp(AmBr)+realmin));
    
    ArBm=A_rad*abs(B_mid);
    ArBm=succ_near(ArBm+((n+2)*u*ufp(ArBm)+realmin));
    
    ArBr=A_rad*B_rad;
    ArBr=succ_near(ArBr+((n+2)*u*ufp(ArBr)+realmin));
    
    rad_sum=C_rad+AmBr+ArBm+ArBr;
    clear AmBr ArBm ArBr
    
    C_rad = succ_near((rad_sum)+3*u*ufp(rad_sum));

    clear rad_sum
    
end

function S = ufp(P)
    
    u=2^-53;
    a=2^52+1;
    Q=a*P;
    T=(1-u)*Q;
    S=abs(Q-T);

end

function [succ] = succ_near(c)

eta=2^-1074;
u=2^-53;
phi=u*(1+2*u);
if abs(c)>=(1/2)*u^-2*eta
e=phi*abs(c);
succ=c+e;
elseif abs(c)<u^-1*eta
succ=c+eta;
else
C=u^-1*c;
e=phi*abs(C);
succ=(C+e)*u;
end
end




:mkl